In [ ]:
%pip install deepface

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------------------------------- 1.3/1.3 MB 23.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------------------------------------- 1.9/1.9 MB 35.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/376.0 MB ? eta -:--:--
   - -------------------------------------- 10.7/376.0 MB 56.1 MB/s eta 0:00:07
   -- ------------------------------------- 19.9/376.0 MB 48.4 MB/s eta 0:00:08
   --- ------------------------------------ 31.2/376.0 MB 49.5 MB/s eta 0:00:07
   ---- ----------------------------------- 44.0/376.0 MB 53.9 MB/s eta 0:00:07
   ------ --------------------------------- 57.7/376.0 MB 54.8 MB/s eta 0:00:06
   ------- -------------------------------- 72.9/376.0 MB 57.4 MB/s eta 0:00:06
   --------- ------------------------------ 86.8/376.0 

  You can safely remove it manually.


In [2]:
pip install tf-keras

   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 1.7/1.7 MB 18.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import json
from deepface import DeepFace
import cv2
import os
import numpy as np
from datetime import datetime

# Helper function to safely convert NumPy types to standard Python types
def convert(obj):
    if isinstance(obj, np.float32) or isinstance(obj, np.float64):
        return float(obj)
    return obj

# Save results with a timestamped filename
def save_results(info, prefix="analysis_result"):
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{prefix}_{timestamp}.json"
    with open(filename, 'w') as f:
        json.dump(info, f, indent=4)
    print(f"\nAnalysis saved to {filename}")

# Analyze a still image from file path
def analyze_image(image_path):
    if not os.path.exists(image_path):
        print(f"Error: {image_path} not found.")
        return
    
    img = cv2.imread(image_path)
    cv2.imshow("Input Image", img)
    cv2.waitKey(1)

    print("Analyzing face, please wait...")
    result = DeepFace.analyze(img_path=image_path, actions=['age', 'gender', 'emotion', 'race'], enforce_detection=False)

    if isinstance(result, list):
        result = result[0]

    analyzed_info = {
        "Age": convert(result.get('age')),
        "Gender": {k: convert(v) for k, v in result.get('gender', {}).items()},
        "Emotion": result.get('emotion'),
        "Race": result.get('race')
    }

    print("\nAnalysis Result:")
    for key, value in analyzed_info.items():
        print(f"{key}: {value}")

    save_results(analyzed_info, prefix="image_analysis")
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Analyze live webcam feed
def analyze_live_camera():
    cap = cv2.VideoCapture(0)
    print("Press 's' to capture image for analysis or 'q' to quit.")

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to access camera.")
            break

        cv2.imshow("Live Feed - Press 's' to Scan", frame)
        key = cv2.waitKey(1) & 0xFF

        if key == ord('s'):
            img = frame.copy()
            cap.release()
            cv2.destroyAllWindows()

            cv2.imshow("Captured Image", img)
            cv2.waitKey(1)
            print("Analyzing captured face, please wait...")

            result = DeepFace.analyze(img_path=img, actions=['age', 'gender', 'emotion', 'race'], enforce_detection=False)

            if isinstance(result, list):
                result = result[0]

            analyzed_info = {
                "Age": convert(result.get('age')),
                "Gender": {k: convert(v) for k, v in result.get('gender', {}).items()},
                "Emotion": result.get('emotion'),
                "Race": result.get('race')
            }

            print("\nAnalysis Result:")
            for key, value in analyzed_info.items():
                print(f"{key}: {value}")

            save_results(analyzed_info, prefix="live_analysis")
            break

        elif key == ord('q'):
            print("Exiting...")
            break

    cap.release()
    cv2.destroyAllWindows()

# MAIN MENU
print("Face Analysis Tool\nChoose an input method:")
print("1. Upload Image")
print("2. Live Webcam Scan")

choice = input("Enter 1 or 2: ").strip()

if choice == '1':
    path = input("Enter full path to the image: ").strip()
    analyze_image(path)
elif choice == '2':
    analyze_live_camera()
else:
    print("Invalid input. Please restart and choose 1 or 2.")


Face Analysis Tool
Choose an input method:
1. Upload Image
2. Live Webcam Scan


Enter 1 or 2:  2


Press 's' to capture image for analysis or 'q' to quit.
Analyzing captured face, please wait...


Action: race: 100%|██████████████████████████████████████████████████████████████████████| 4/4 [00:05<00:00,  1.41s/it]


Analysis Result:
Age: 30
Gender: {'Woman': 74.38170313835144, 'Man': 25.618290901184082}
Emotion: {'angry': 0.06586453920726712, 'disgust': 1.7010172418648526e-05, 'fear': 28.79654936769899, 'happy': 6.938313361305795, 'sad': 0.381645373464845, 'surprise': 13.317757148608992, 'neutral': 50.49985352157673}
Race: {'asian': 9.495735526593265, 'indian': 15.83215107534515, 'black': 4.87755963175778, 'white': 13.555520647106093, 'middle eastern': 12.241563976816657, 'latino hispanic': 43.99746727973613}

Analysis saved to live_analysis_20250504_145940.json
